# ETH HMA 策略分析 - 数据加载与预处理

本notebook用于加载和预处理ETH历史数据，为后续的策略分析做准备。

## 功能说明
- 加载ETH历史数据（1h和4h级别）
- 计算HMA技术指标
- 数据质量检查和清洗
- 为策略分析准备数据


In [ ]:
# 导入必要的库
import sys
import os
sys.path.append('../src')

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
from pathlib import Path
import warnings
warnings.filterwarnings('ignore')

# 设置中文字体支持
plt.rcParams['font.sans-serif'] = ['Microsoft YaHei', 'SimHei', 'Arial Unicode MS']
plt.rcParams['axes.unicode_minus'] = False
plt.rcParams['font.size'] = 12

# 设置pandas显示选项
pd.set_option('display.max_columns', None)
pd.set_option('display.width', None)
pd.set_option('display.max_colwidth', None)

print("✅ 库导入完成")


In [ ]:
# 导入项目模块
from managers.project_manager import ProjectManager
from analyzers.trend_analyzer import TrendAnalyzer
from utils.config import *

print("✅ 项目模块导入完成")


In [ ]:
# 初始化项目管理器
project_manager = ProjectManager()

# 检查数据文件
data_dir = Path("../src/utils/data")
data_files = list(data_dir.glob("*.parquet"))
print(f"📁 找到 {len(data_files)} 个数据文件:")
for file in data_files:
    print(f"  - {file.name}")


In [ ]:
# 加载4h级别数据（主要分析目标）
data_file = data_dir / "ETHUSDT_4h_processed_20250926_214833.parquet"
if data_file.exists():
    df_4h = pd.read_parquet(data_file)
    print(f"✅ 4h数据加载成功: {len(df_4h)} 条记录")
    print(f"📅 时间范围: {df_4h.index[0]} 到 {df_4h.index[-1]}")
else:
    print("❌ 4h数据文件不存在，请先运行数据收集脚本")
    df_4h = None


In [ ]:
# 数据概览
if df_4h is not None:
    print("📊 4h数据概览:")
    print(f"数据形状: {df_4h.shape}")
    print(f"\n列名: {list(df_4h.columns)}")
    print(f"\n数据类型:")
    print(df_4h.dtypes)
    print(f"\n前5行数据:")
    display(df_4h.head())


In [ ]:
# 检查HMA指标
if df_4h is not None:
    hma_cols = [col for col in df_4h.columns if 'HMA' in col]
    print(f"🔍 HMA指标列: {hma_cols}")
    
    if hma_cols:
        print(f"\nHMA指标统计:")
        display(df_4h[hma_cols].describe())
    else:
        print("❌ 未找到HMA指标，请检查数据处理")
        
    # 检查缺失值
    print(f"\n缺失值统计:")
    missing_data = df_4h.isnull().sum()
    print(missing_data[missing_data > 0])


In [ ]:
# 保存处理后的数据到全局变量
if df_4h is not None:
    # 确保数据按时间排序
    df_4h = df_4h.sort_index()
    
    # 保存到全局变量
    globals()['df_4h'] = df_4h
    
    print("✅ 4h数据准备完成，可用于后续分析")
    print(f"📈 价格范围: ${df_4h['close'].min():.2f} - ${df_4h['close'].max():.2f}")
    print(f"📅 分析期间: {df_4h.index[0].strftime('%Y-%m-%d')} 到 {df_4h.index[-1].strftime('%Y-%m-%d')}")
else:
    print("❌ 数据加载失败，请检查数据文件")
